Задание 1:

Реализуйте класс, представляющий собой матрицу, который можно
множать слева и справа на одномерные numpy вектора за время O(m),
где m – количество ненулевых элементов матрицы.

- 1б Только левое/правое умножение.
- 2б Левое и правое умножение.


In [1]:
import numpy as np


class SparseMatrix:
    
    def __init__(self, matrix: np.array):
        nonzero_indexes = np.nonzero(matrix)
        self.n_rows, self.n_columns = matrix.shape
        self.value_arr = matrix[nonzero_indexes]
        self.column_index_arr = nonzero_indexes[1]
        # первый элемент равен 0; каждый следующий на i-той позиции равен 
        # количеству ненулевых элементов матрицы во всех строчках от нулевой до (i - 1) включительно
        self.row_indexing_arr = []
        for i in range(matrix.shape[0] + 1):
            curr_sum = 0
            for j in nonzero_indexes[0]:
                if j < i:
                    curr_sum += 1
            self.row_indexing_arr.append(curr_sum)
    
    # x - вектор; будем передавать в виде строчки
    # matrix @ x
    def right_mul(self, x: np.array):
        ans = np.zeros(self.n_rows)
        last_index = 0
        for i in range(1, len(self.row_indexing_arr)):
            elem_count = self.row_indexing_arr[i] - self.row_indexing_arr[i - 1]
            for j in range(last_index, last_index + elem_count):
                ans[i - 1] += self.value_arr[j] * x[self.column_index_arr[j]]
            last_index += elem_count
        return ans
    
    # x.T @ matrix
    def left_mul(self, x: np.array):
        ans = np.zeros(self.n_columns)
        last_index = 0
        for i in range(1, len(self.row_indexing_arr)):
            elem_count = self.row_indexing_arr[i] - self.row_indexing_arr[i - 1]
            for j in range(last_index, last_index + elem_count):
                ans[self.column_index_arr[j]] += self.value_arr[j] * x[i - 1]
            last_index += elem_count
        return ans
    

In [2]:
matrix = np.array([[1, 2, 0],
                   [0, 4, 0],
                   [0, 2, 6]
                  ])
sparse_matrix = SparseMatrix(matrix)
x1, x2 = np.array([1, 1, 1]), np.array([2, 4, 1])
assert(np.array_equal(sparse_matrix.right_mul(x1), [3, 4, 8]))
assert(np.array_equal(sparse_matrix.right_mul(x2), [10, 16, 14]))
assert(np.array_equal(sparse_matrix.left_mul(x1), [1, 8, 6]))
assert(np.array_equal(sparse_matrix.left_mul(x2), [2, 22, 6]))

Задание 2:

- 3б. Реализуйте метод сопряженных градиентов для минимизации
$||Ax − b||$
таким образом, чтобы его сложность была порядка O(nm), где n –
размерность x, m – число ненулевых элементов A.

Чтобы получить сложность O(nm), воспользуемся прошлой задачей.

Умеем решать за O(n) задачу минимизации функции
$\frac{1}{2} \cdot <Ax, x> - <b, x>$

Распишем нашу функцию:
$||Ax - b||^2 = (Ax - b)^T \cdot (Ax - b) = x^TA^TAx - x^TA^Tb - b^TAx - b^Tb = <A^TAx, x> - 2<A^Tb, x> - b^Tb$

Тогда наша задача эквивалентна предыдущей, если взять матрицу $A^TA$ и вектор $A^Tb$

In [3]:
def conjugate_gradient_method(A: np.array, b: np.array):
    '''
    Returns: последовательность приближений [x_0, x_1, .., x_m], где m - количество столбцов матрицы A
    '''
    ans = []
    sparse_A = SparseMatrix(A)
    sparse_A_T = SparseMatrix(A.T)
    # new_A = A.T @ A
    n, m = A.shape
    new_A = np.zeros((m, m))
    for i in range(m):
        new_column = sparse_A_T.right_mul(A[:, i])
        new_A[:, i] = new_column
    new_b = sparse_A_T.right_mul(b)
    x_0 = np.zeros(m)
    sparse_new_A = SparseMatrix(new_A)
    curr_r = new_b - sparse_new_A.right_mul(x_0)
    curr_p = curr_r
    curr_x = x_0
    for _ in range(m):
        curr_alpha = curr_r.T @ curr_r / (curr_p.T @ (sparse_new_A.right_mul(curr_p)))
        curr_x = curr_x + curr_alpha * curr_p
        ans.append(curr_x)
        new_r = curr_r - curr_alpha * sparse_new_A.right_mul(curr_p)
        curr_beta = new_r.T @ new_r / (curr_r.T @ curr_r)
        curr_r = new_r
        curr_p = curr_r + curr_beta * curr_p
    return np.array(ans)

In [4]:
import cvxpy as cp


n, m = 7, 5
EPS = 1e-4

for _ in range(100):
    A = np.random.rand(n, m)
    b = np.random.rand(n)

    output = conjugate_gradient_method(A, b)[-1]
    x = cp.Variable(m)
    constraints = []
    obj = cp.Minimize(cp.sum_squares(A @ x - b))
    cp.Problem(obj, constraints).solve()
    expected = x.value

    for i in range(m):
        assert(abs(output[i] - expected[i]) < EPS)